In [5]:
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Scopes allow us to read and write tasks
SCOPES = ['https://www.googleapis.com/auth/tasks']

# Authentication
def authenticate_google_tasks():
    creds = None
    # The token.pickle stores the user's access and refresh tokens
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or expired, get new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('tasks', 'v1', credentials=creds)

# Create a new task list
def create_task_list(service, title):
    tasklist = {
        'title': title
    }
    result = service.tasklists().insert(body=tasklist).execute()
    print(f"Task list created: {result['title']}, ID: {result['id']}")
    return result['id']

# Get all task lists
def get_task_lists(service):
    result = service.tasklists().list().execute()
    tasklists = result.get('items', [])
    if not tasklists:
        print("No task lists found.")
    else:
        for tasklist in tasklists:
            print(f"Task List: {tasklist['title']}, ID: {tasklist['id']}")
    return tasklists

# Create a new task in a specific task list
def create_task_in_list(service, tasklist_id, title, due_date=None):
    task = {
        'title': title,
    }
    if due_date:
        task['due'] = due_date
    result = service.tasks().insert(tasklist=tasklist_id, body=task).execute()
    print(f"Task created in list: {result['title']}")

# Get tasks from a specific task list
def get_tasks_from_list(service, tasklist_id):
    results = service.tasks().list(tasklist=tasklist_id).execute()
    tasks = results.get('items', [])
    if not tasks:
        print("No tasks found.")
    else:
        for task in tasks:
            print(f"Task: {task['title']}, Due: {task.get('due', 'No due date')}")

# Delete a task by task ID
def delete_task(service, tasklist_id, task_id):
    service.tasks().delete(tasklist=tasklist_id, task=task_id).execute()
    print("Task deleted.")

# Main execution with an example of creating and managing tasks
if __name__ == '__main__':
    service = authenticate_google_tasks()

    # Create a new task list called "Schule"
    schule_tasklist_id = "T2tteVU3OWdwVnQ2Q3pfNw"
 
    # Create a new task list called "Schule" if it doesn't exist
    create_task_in_list(service, schule_tasklist_id, "Study for exam", "2024-10-15T12:00:00.000Z")
    # Get tasks from "Schule" task list
    print("\nTasks in 'Schule' task list:")
    get_tasks_from_list(service, schule_tasklist_id)

    # Optionally, you can delete a task by its ID (you'll need to find the task ID from the printed list)
    # Example (replace 'TASK_ID' with actual task ID):
    # delete_task(service, schule_tasklist_id, 'TASK_ID')


Task created in list: Study for exam

Tasks in 'Schule' task list:
Task: Study for exam, Due: 2024-10-15T00:00:00.000Z
Task: Study for exam, Due: 2024-10-15T00:00:00.000Z
Task: Passed: testtest, Due: 2024-10-03T00:00:00.000Z
Task: Passed: test, Due: 2024-10-03T00:00:00.000Z
Task: Passed: hiiiiii, Due: 2024-10-03T00:00:00.000Z


In [3]:
print("\nTasks in 'Schule' task list:")
get_tasks_from_list(service, schule_tasklist_id)


Tasks in 'Schule' task list:
Task: Study for exam, Due: 2024-10-15T00:00:00.000Z


In [6]:
from datetime import datetime, timedelta, timezone
import time

# Flag tasks as "Passed" after their due date, and mark them as "Completed" one day later
def process_tasks(service, tasklist_id):
    results = service.tasks().list(tasklist=tasklist_id).execute()
    tasks = results.get('items', [])
    current_time = datetime.now(timezone.utc).date()  # Use timezone-aware datetime object, but only compare the date

    for task in tasks:
        # Check if the task has a due date
        if 'due' in task:
            # Convert the task due date to a timezone-aware datetime object
            task_due_date = datetime.strptime(task['due'], "%Y-%m-%dT%H:%M:%S.%fZ").date()
            
            # Check if the task's due date has passed
            if task_due_date < current_time:
                # Add the "Passed" flag if it hasn't already been added
                if not task['title'].startswith('Passed'):
                    task['title'] = f"Passed: {task['title']}"
                    service.tasks().update(tasklist=tasklist_id, task=task['id'], body=task).execute()
                
                # Check if more than a day has passed since the due date and mark it as completed
                if current_time > task_due_date + timedelta(days=1) and task['status'] != 'completed':
                    task['status'] = 'completed'
                    service.tasks().update(tasklist=tasklist_id, task=task['id'], body=task).execute()

# Main loop to automate the process (check every second)
def monitor_tasks(service, tasklist_id):
    while True:
        process_tasks(service, tasklist_id)
        time.sleep(1)  # Check every second

# Example usage:
if __name__ == '__main__':
    service = authenticate_google_tasks()

    # Assuming you want to monitor the "Schule" task list
    schule_tasklist_id = "T2tteVU3OWdwVnQ2Q3pfNw"

    monitor_tasks(service, schule_tasklist_id)


In [6]:
from datetime import datetime, timezone
import time
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Scopes allow us to read and write tasks
SCOPES = ['https://www.googleapis.com/auth/tasks']

# Authentication
def authenticate_google_tasks():
    creds = None
    # The token.pickle stores the user's access and refresh tokens
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or expired, get new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('tasks', 'v1', credentials=creds)

# Get Task List ID by Task List Title
def get_tasklist_id_by_title(service, title):
    result = service.tasklists().list().execute()
    tasklists = result.get('items', [])
    for tasklist in tasklists:
        if tasklist['title'].lower() == title.lower():
            return tasklist['id']
    raise ValueError(f"Tasklist '{title}' not found")

# Get Task ID by Task Title in a specific Task List
def get_task_id_by_title(service, tasklist_id, title):
    result = service.tasks().list(tasklist=tasklist_id).execute()
    tasks = result.get('items', [])
    for task in tasks:
        if task['title'].lower() == title.lower():
            return task['id']
    raise ValueError(f"Task '{title}' not found in task list with ID '{tasklist_id}'")

# Create a Task
def create_task(service, tasklist_id, title, due_date=None, priority=None, description=None):
    task_body = {
        'title': title
    }
    # Add optional parameters if provided
    if due_date:
        task_body['due'] = due_date
    if priority:
        task_body['notes'] = f"Priority: {priority}"  # Notes used for priority in Google Tasks
    if description:
        task_body['notes'] = (task_body['notes'] + f"\nDescription: {description}") if 'notes' in task_body else f"Description: {description}"

    task = service.tasks().insert(tasklist=tasklist_id, body=task_body).execute()
    print(f"Created task '{title}' with ID: {task['id']}")
    return task['id']

# Update a Task (optional params: title, due date, priority, description)
def update_task(service, tasklist_id, task_id, title=None, due_date=None, priority=None, description=None):
    task = service.tasks().get(tasklist=tasklist_id, task=task_id).execute()

    # Update fields only if new values are provided
    if title:
        task['title'] = title
    if due_date:
        task['due'] = due_date
    if priority or description:
        notes = task.get('notes', "")
        if priority:
            notes = f"Priority: {priority}"
        if description:
            notes += f"\nDescription: {description}"
        task['notes'] = notes

    updated_task = service.tasks().update(tasklist=tasklist_id, task=task_id, body=task).execute()
    print(f"Updated task '{task_id}'")
    return updated_task

# Get all tasks from a specific task list
def get_tasks(service, tasklist_id):
    result = service.tasks().list(tasklist=tasklist_id).execute()
    tasks = result.get('items', [])
    if not tasks:
        print(f"No tasks found in task list with ID '{tasklist_id}'")
        return []
    for task in tasks:
        print(f"Task: {task['title']} | Due Date: {task.get('due', 'No due date')} | Notes: {task.get('notes', 'No notes')}")
    return tasks

# Example usage:
if __name__ == '__main__':
    service = authenticate_google_tasks()

    # Example: Get task list ID by title (you can replace "Schule" with any list name)
    tasklist_id = get_tasklist_id_by_title(service, "Schule")

    # Example: Create a task
    create_task(service, tasklist_id, title="Finish Homework", due_date="2024-10-05T00:00:00.000Z", priority="High", description="Finish the math homework by this weekend")
    time.sleep(20)  # Wait for the task to be created before updating
    # Example: Update a task by getting the task ID from the task title
    task_id = get_task_id_by_title(service, tasklist_id, "Finish Homework")
    update_task(service, tasklist_id, task_id, due_date="2024-10-06T00:00:00.000Z", priority="Medium", description="Updated description for the task")

    # Example: Get all tasks
    get_tasks(service, tasklist_id)


Created task 'Finish Homework' with ID: d3BmMzN6a0VWbnUwQUNQUA
Updated task 'd3BmMzN6a0VWbnUwQUNQUA'
Task: Finish Homework | Due Date: 2024-10-06T00:00:00.000Z | Notes: Priority: Medium
Description: Updated description for the task
Task: Passed: testtest | Due Date: 2024-10-03T00:00:00.000Z | Notes: No notes
Task: Passed: test | Due Date: 2024-10-03T00:00:00.000Z | Notes: No notes
Task: Passed: hiiiiii | Due Date: 2024-10-03T00:00:00.000Z | Notes: No notes
